In [3]:
!pip install --upgrade transformers

In [2]:
!pip install -q transformers datasets scikit-learn pandas numpy torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 794.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.8 MB/s eta 0:00:00


In [4]:
from google.colab import files
uploaded = files.upload()

Saving key-sentences.csv to key-sentences.csv


In [5]:
import pandas as pd

df = pd.read_csv(next(iter(uploaded)), encoding='latin1')
df = df[["messages", "target"]].dropna()
df = df.rename(columns={"messages": "text", "target": "labels"})
df.head()



,text,labels
0,****head,1
1,100 pu$$is,1
2,100 pu$$ys,1
3,2 girls 1 cup,1
4,2g1c,1


In [6]:
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["text"].tolist(),
    df["labels"].tolist(),
    test_size=0.2,
    random_state=42
)

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [7]:
import torch

class AbuseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = AbuseDataset(train_encodings, train_labels)
val_dataset = AbuseDataset(val_encodings, val_labels)


In [8]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    run_name="abuse_detection_experiment",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# import os
# os.environ["WANDB_DISABLED"] = "false"


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: webhost0987654321 (webhost0987654321-sanjivani-college-of-engineering) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.129600
1000,0.066000
1500,0.029300
2000,0.005200
2500,0.010900


TrainOutput(global_step=2502, training_loss=0.04814396837269228, metrics={'train_runtime': 429.7356, 'train_samples_per_second': 46.536, 'train_steps_per_second': 5.822, 'total_flos': 212133602678184.0, 'train_loss': 0.04814396837269228, 'epoch': 3.0})

In [10]:
model.save_pretrained("./abuse_detection_model")
tokenizer.save_pretrained("./abuse_detection_model")


('./abuse_detection_model/tokenizer_config.json',
 './abuse_detection_model/special_tokens_map.json',
 './abuse_detection_model/vocab.txt',
 './abuse_detection_model/added_tokens.json',
 './abuse_detection_model/tokenizer.json')

In [11]:
!zip -r abuse_detection_model.zip abuse_detection_model/

  adding: abuse_detection_model/ (stored 0%)
  adding: abuse_detection_model/model.safetensors (deflated 8%)
  adding: abuse_detection_model/config.json (deflated 45%)
  adding: abuse_detection_model/special_tokens_map.json (deflated 42%)
  adding: abuse_detection_model/vocab.txt (deflated 53%)
  adding: abuse_detection_model/tokenizer.json (deflated 71%)
  adding: abuse_detection_model/tokenizer_config.json (deflated 75%)


In [13]:
from transformers import pipeline

classifier = pipeline("text-classification", model="./abuse_detection_model", tokenizer="./abuse_detection_model")

print(classifier("You are big dick head"))
print(classifier("You are so helpful, thank you!"))


Device set to use cuda:0


[{'label': 'LABEL_1', 'score': 0.9998434782028198}]
[{'label': 'LABEL_0', 'score': 0.9999414682388306}]


In [ ]:
from google.colab import files

# Assuming 'my_file.txt' is the file you want to download
files.download('abuse_detection_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
!pip freeze > requirements.txt
